In [1]:
!pip install category_encoders

In [2]:
!pip install yellowbrick

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from category_encoders import TargetEncoder, OneHotEncoder
from yellowbrick.regressor import ResidualsPlot
%matplotlib inline

In [3]:
!pip3 install google-colab

  Using cached google_colab-1.0.0-py2.py3-none-any.whl
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached portpicker-1.2.0-py3-none-any.whl
  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached tornado-4.5.3-cp39-cp39-macosx_11_0_arm64.whl
  Using cached pyasn1_modules-0.3.0-py2.py3-none-any.whl (181 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Using cached cachetools-5.3.0-py3-none-any.whl (9.3 kB)
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached simplegeneric-0.8.1-py3-none-any.whl
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)


  Using cached jupyter_client-7.3.5-py3-none-any.whl (132 kB)
  Using cached jupyter_client-7.3.4-py3-none-any.whl (132 kB)
  Using cached jupyter_client-7.3.3-py3-none-any.whl (131 kB)
  Using cached jupyter_client-7.3.2-py3-none-any.whl (131 kB)
  Using cached jupyter_client-7.3.1-py3-none-any.whl (130 kB)
  Using cached jupyter_client-7.3.0-py3-none-any.whl (130 kB)
  Using cached jupyter_client-7.2.2-py3-none-any.whl (130 kB)
  Using cached jupyter_client-7.2.1-py3-none-any.whl (130 kB)
  Using cached jupyter_client-7.2.0-py3-none-any.whl (130 kB)
  Using cached jupyter_client-7.1.2-py3-none-any.whl (130 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1049 lines of output]
      /private/var/folders/88/9yj6v4297lgdsq091dy85pd40000gn/T/pip-install-hz9f89kd/pandas_53b0dadd1347456ca4cb84f242192e10/setup.py:435: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
    

Failed to build pandas
ERROR: Could not build wheels for pandas, which is required to install pyproject.toml-based projects


# Exploratory Data Analysis

In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
data_dir = '/content/drive/MyDrive/realtor-data.csv'

In [ ]:
data = pd.read_csv(data_dir)
data.head()

In [ ]:
data.dtypes

In [ ]:
# duplicate values
data.duplicated().sum()

In [ ]:
# drop duplicate values
data.drop_duplicates(inplace=True)

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
# drop columns that won't be used
data = data.drop(["status", "acre_lot", "full_address", "street", "sold_date"], axis=1)

In [ ]:
# missing values
data.isnull().sum()

In [ ]:
# drop missing values
data.dropna(how='any', inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

In [ ]:
# remove outlier (greater than 3 standard deviations)
data = data[np.abs(stats.zscore(data['price'])) < 3]
data = data[np.abs(stats.zscore(data['bed'])) < 3]
data = data[np.abs(stats.zscore(data['bath'])) < 3]
data = data[np.abs(stats.zscore(data['house_size'])) < 3]
data.shape

In [ ]:
data.describe()

In [ ]:
sns.set(rc={'figure.figsize':(12, 9)})
sns.scatterplot(data = data, x='house_size', y='price')

In [ ]:
sns.distplot(data.price)

In [ ]:
sns.barplot(x=data.bed , y=data.price)

In [ ]:
sns.barplot(x=data.bath , y=data.price)

In [ ]:
sns.countplot(data.state)
plt.xticks(rotation=90)

In [ ]:
data[data.state=='Virgin Islands'].price.mean()

In [ ]:
data.groupby("state")["price"].mean().round(2).plot(kind="bar")

In [ ]:
# Virgin Islands has very less data but a high average price.
# West Virginia has very less data but a low average price.
# drop data in these two states
data = data[data.state != 'Virgin Islands']
data = data[data.state != 'West Virginia']

In [ ]:
sns.countplot(data.state)
plt.xticks(rotation=90)

In [ ]:
data.groupby("state")["price"].mean().plot(kind='bar', y='price')

In [ ]:
data.head()

In [ ]:
data["zip_code"] = data["zip_code"].apply(str)

In [ ]:
# target encoding
data["city_enc"] = TargetEncoder().fit_transform(data["city"], data["price"])
data["zip_enc"] = TargetEncoder().fit_transform(data["zip_code"], data["price"])

In [ ]:
# one hot encoding
states_encoded = OneHotEncoder(use_cat_names=True).fit_transform(data["state"])
data = pd.concat([data, states_encoded], axis=1)

In [ ]:
data = data.drop(["city","state","zip_code"], axis=1)
data.head()

In [ ]:
data.shape

In [ ]:
# file_path= '/content/drive/MyDrive/data_cleaned.csv'
# data.to_csv(file_path)

# Initial Model Training

In [ ]:
from sklearn.linear_model import LinearRegression  
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error ,mean_squared_error, median_absolute_error,confusion_matrix,accuracy_score,r2_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from xgboost import XGBRegressor

In [ ]:
X = data.drop(columns="price")           
y = data["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print("X Train : ", X_train.shape)
print("X Test  : ", X_test.shape)
print("Y Train : ", y_train.shape)
print("Y Test  : ", y_test.shape)

In [ ]:
LR = LinearRegression()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
print("Linear Regression MSE:", mean_squared_error(y_test, y_pred))
print("Linear Regression R2:", r2_score(y_test, y_pred))